In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import open3d as o3d

# First - let's try to do this in 2D

Steps:
- Create a set of lines with their normals
- Calculate the intersections (within epsilon neighborhoods)
- Sort the intersection points based on their x-coords
- Plot the result in 2D

In [91]:
class Line:
    def __init__(self, m, b, xlimits = [0,1], resolution = 31, y = np.empty((1,1))):
        """ The normal is of the form [a,b,c], where ax + by + c = 0. """
        self.m = m
        self.b = b
        self.xlimits = xlimits
        self.N = resolution
        
        # Create some x-values to help define the line
        self.x = np.linspace(self.xlimits[0], self.xlimits[1], self.N)
        
        # Get the grid step size
        self.step = self.x[2] - self.x[1]
        
        # Set epsilon as half the diagonal Euclidean distance between 2D grid points
        self.epsilon = np.sqrt(self.step**2 + self.step**2)/2
        
        self.frame = pd.DataFrame(self.x, columns = ['x'])
        
        if not y.size:
            self.frame['y'] = self.m*y + self.b*np.ones(self.N)
        
        else:
            self.frame['y'] = self.m*self.x + self.b*np.ones(self.N)
            
    def activate(self):
        """ Apply element-wise ReLU """
        self.frame['y'] = [np.max([0, i]) for i in self.frame['y']]
            
    def intersect(self, other):
        """ Determine points of intersection between two lines """
        
        cross = pd.merge(self.frame, other.frame, how = 'cross', suffixes = ['1','2'])
        
        # Get the pointwise Euclidean distance between planes
        cross['distance'] = np.sqrt((cross['x1'] - cross['x2'])**2 + 
                                    (cross['y1'] - cross['y2'])**2)
        
        # Find the points within epsilon neighborhoods of each other
        close_points = cross[cross['distance'] < self.epsilon]
        
        # This part is kind of arbitrary - take the left point
        result = close_points[['x1','y1']]
        
        return result.rename(columns = {'x1': 'x', 'y1': 'y'})
    
    def plot(self):
        fig = px.scatter(self.frame, x = 'x', y = 'y')
        
        fig.update_traces(marker = dict(size = 2, 
                                        line = dict(width = 2, 
                                                    color = 'DarkSlateGrey')),
                          selector = dict(mode = 'markers'))
        
        fig.show()

In [92]:
# First layer parameters
m11 = np.random.randn()
m21 = np.random.randn()
b11 = np.random.randn()
b21 = np.random.randn()

# Second layer parameters
m12 = np.random.randn()
m22 = np.random.randn()
b12 = np.random.randn()
b22 = np.random.randn()

In [93]:
line1 = Line(m = m11, b = b11)
line2 = Line(m = m21, b = b21)

In [94]:
line1.activate()
line2.activate()

In [95]:
line3 = Line(m = m12, b = b12, y = line1.frame['y'])
line4 = Line(m = m22, b = b22, y = line2.frame['y'])

In [98]:
line3.activate()
line4.activate()

In [99]:
line3.plot()

In [100]:
line4.plot()

In [44]:
line1 = Line(m = -2, b = 1)
line2 = Line(m = 1, b = 1)

In [2]:
point_cloud = np.loadtxt('sample.xyz',skiprows=1)
point_cloud[:10]

array([[  2.75421051,  -4.75981225,  -6.44403559,   8.        ,
         12.        ,  13.        ],
       [  1.07206253,  -0.7416798 ,  -5.10408789,  14.        ,
         12.        ,  17.        ],
       [ -2.84094568,  -1.76128496,  -6.89372498,  14.        ,
         19.        ,  25.        ],
       [ -2.4484963 ,   0.49504791,  -4.2454682 ,  96.        ,
        105.        ,  74.        ],
       [  2.96957211,  -4.67333497,  -6.42598539,  27.        ,
         33.        ,  31.        ],
       [  4.93840889,  -4.73994435,  -7.98122549,  46.        ,
         47.        ,  41.        ],
       [  4.39719013,  -2.57718195,  -7.33467585,  12.        ,
         12.        ,  14.        ],
       [  2.25035099,  -1.21843304,  -6.33910852,  38.        ,
         39.        ,  31.        ],
       [  3.12998489,  -4.83957614,  -6.24208885,  32.        ,
         34.        ,  33.        ],
       [  3.25234417,  -4.89076139,  -6.10257154,  27.        ,
         28.        ,  23. 

In [39]:
def generate_plane(normal, frame):
    """ Take in an array representing the normal of an affine plane (with bias)
        and return all (x,y,z) triples that define the plane.
        
        args:
        -----
        normal - np array
    """
    
    a, b, c, d = normal[:]
        
    frame['z'] = -(a*frame['x'] + b*frame['y'] + d*np.ones(N*N))/c
    
    return frame

In [217]:
class Plane:
    def __init__(self, normal, xlimits = [0,1], ylimits = [0,1], resolution = 51):
        self.normal = normal
        self.xlimits = xlimits
        self.ylimits = ylimits
        self.N = resolution
        
        x = np.linspace(self.xlimits[0], self.xlimits[1], self.N)
        y = np.linspace(self.ylimits[0], self.ylimits[1], self.N)
        
        # Get the step size
        self.xstep = x[2] - x[1]
        self.ystep = y[2] - y[1]
        
        # Set epsilon as the diagonal Euclidean distance between grid points
        self.epsilon = np.sqrt(self.xstep**2 + self.ystep**2)/2
        
        self.frame = pd.DataFrame(itertools.product(x, y), columns = ['x', 'y'])
        a,b,c,d = self.normal[:]
        
        try:
            self.frame['z'] = -(a*self.frame['x'] + b*self.frame['y'] + d*np.ones(self.N*self.N))/c
            
        except:
            self.frame['z'] = np.zeros(self.N*self.N)       
            
    def intersect(self, other):
        """ Compare points in one plane to points in another plane. 
        
            If the points are within an epsilon neighborhood of each other,
            then create a vertex at the common point (point of intersection).
        """
        
        cross = pd.merge(self.frame, other.frame, how = 'cross', suffixes = ['1','2'])
        
        # Get the pointwise Euclidean distance between planes
        cross['distance'] = np.sqrt((cross['x1'] - cross['x2'])**2 + 
                                    (cross['y1'] - cross['y2'])**2 + 
                                    (cross['z1'] - cross['z2'])**2)
        
        # Find the points within epsilon neighborhoods of each other
        close_points = cross[cross['distance'] < self.epsilon]
        
        # This part is kind of arbitrary - take the left point
        result = close_points[['x1','y1','z1']]
        
        return result.rename(columns = {'x1': 'x', 'y1': 'y', 'z1': 'z'})
    
    def plot(self):
        fig = px.scatter_3d(self.frame, x = 'x', y = 'y', z = 'z')
        
        fig.update_traces(marker = dict(size = 2, 
                                        line = dict(width = 2, 
                                                    color = 'DarkSlateGrey')),
                          selector = dict(mode = 'markers'))
        
        fig.show()

In [218]:
n1 = np.array([1,0,1,1])
n2 = np.array([-1,2,1,1])

p1 = Plane(n1, resolution = 11)
p2 = Plane(n2, resolution = 11)

In [219]:
h = p1.intersect(p2)

In [220]:
p1.plot()

In [170]:
import plotly.express as px

fig = px.scatter_3d(h, x='x', y='y', z='z')

fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show()

In [172]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(x=p1.frame['x'], y=p1.frame['y'], z=p1.frame['z'],
                                   mode='markers')])

fig.add_trace(go.Scatter3d(x=p2.frame['x'], y=p2.frame['y'], z=p2.frame['z'],
                                   mode='markers'))

fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

# This part is all to create and load a 3d mesh of an object

**Note:** the Poisson mesh doesn't work very well for planar data.

In [173]:
import open3d as o3d

In [184]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(p1.frame[['x','y','z']].to_numpy())

In [199]:
print("Recompute the normal of the downsampled point cloud")
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=3))
o3d.visualization.draw_geometries([pcd],
                                  point_show_normal=True)

Recompute the normal of the downsampled point cloud


In [200]:
o3d.visualization.draw_geometries([pcd])

In [201]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3*avg_dist

In [202]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [203]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

In [204]:
o3d.io.write_triangle_mesh("p_mesh_c.ply", p_mesh_crop)

True

In [205]:
def lod_mesh_export(mesh, lods, extension):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh("lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [206]:
my_lods2 = lod_mesh_export(poisson_mesh, [8000,800,300], ".ply")

generation of 300 LoD successful


In [207]:
o3d.visualization.draw_geometries([my_lods2[300]])